In [43]:
import numpy as np
from scipy import sparse
import pandas as pd
import pickle
import faiss

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import label_ranking_loss
from sklearn.linear_model import SGDClassifier

from tqdm import tqdm_notebook

import mmh3

In [4]:
x = sparse.load_npz('./x_norm_sparse.npz')
y = sparse.load_npz('./y.npz')

x = np.array(x.todense())
y = np.array(y.todense())
allSmiles = pd.read_csv('allSmiles.csv', header=None)
targetNames = pd.read_csv('targetNames.csv', header=None)

In [18]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
mol = Chem.MolFromSmiles(allSmiles.iloc[0][0])

In [40]:
fp_length = 50
fps_array = list()
for smi in tqdm_notebook(allSmiles[0]):
    mol = Chem.MolFromSmiles(smi)
    fp = AllChem.GetMorganFingerprint(mol, 3)
    fp_values = fp.GetNonzeroElements()
    fp_array = np.zeros(50)
    for key, item in fp_values.items():
        mmhash = mmh3.hash(str(key))%fp_length
        fp_array[mmhash]=item
    fps_array.append(fp_array)


RDKit WARNING: [14:09:43] Conflicting single bond directions around double bond at index 17.
RDKit WARNING: [14:09:43]   BondStereo set to STEREONONE and single bond directions set to NONE.


In [41]:
fps_array = np.array(fps_array)

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     fps_array, y, test_size=0.1, random_state=42)

In [59]:
target_index=0
clf = SGDClassifier(loss='log')
clf.fit(X_train, y_train[:,target_index])

probs = clf.predict_proba(X_test)[:,1]
label_ranking_loss(y_test[:,target_index].reshape(1,-1), probs.reshape(1,-1))

0.1310627631716839

In [57]:
target_index=0
clf = BernoulliNB()
clf.fit(X_train, y_train[:,target_index])

probs = clf.predict_proba(X_test)[:,1]
label_ranking_loss(y_test[:,target_index].reshape(1,-1), probs.reshape(1,-1))

0.15489346846601137

In [65]:
class VirtualScreeningBootstrapperMeans():
    def __init__(self, x, y, clf, weighting=False):
        self.x = x
        self.y = y
        self.clf = clf
        
        self.positive_indices = None
        self.negative_indices = None
        self.faiss_index = None
        self.targetIndex = None
        self.fraction = None
        self.weights = None
        self.weighting=weighting
        
    def setWeighting(self, weighting):
        self.weighting = weighting

    def buildIndex(self, input_x):
        faiss_index = faiss.IndexFlatL2(input_x.shape[1])   # build the index
        faiss_index.add(input_x.astype('float32'))   # add vectors to the index    
        
        self.faiss_index = faiss_index
        
    def setTarget(self, target_index):
        self.targetIndex = target_index
        self.positive_indices = np.where(self.y[:,self.targetIndex]==1)[0]
        self.negative_indices = np.where(self.y[:,self.targetIndex]==0)[0]
        self.weights = np.ones(len(self.positive_indices))
        
        self.buildIndex(self.x[self.positive_indices])
        
    def maskNN(self):
        #normed_weights = normalize(self.weights.reshape(1,-1), norm='l1')[0] #equal weighting
        
        #selection = np.random.choice(self.positive_indices, p = normed_weights) #take a random true positive ligand
        selection = np.random.choice(self.positive_indices) #take a random true positive ligand
        _, knn = self.faiss_index.search(self.x[selection].reshape(1,-1).astype('float32'),
                        int(self.fraction*len(self.positive_indices))) #find it's k-percent nearest neighbors
    
        #if self.weighting: #reduce the weights of the k nearest neighbors
        #    self.weights[knn] = self.weights[knn]*0.5
        
        mask = np.ones(len(self.x), dtype=bool) #create a mask
        mask[self.positive_indices[knn]]=False #set only the neighbors to False
        return mask
    
    def maskNegs(self, mask):
        neg_indices = np.where(self.y[:,self.targetIndex] ==0)[0]
        neg_selection = np.random.choice(neg_indices, int(self.fraction*len(neg_indices)))
        mask[neg_selection]=False
        return mask
    
    def evaluateFold(self, mask):
        probs = self.clf.predict_proba(self.x[~mask])[:,1] #take probability of test ligands being positive
        ranking_loss = label_ranking_loss(self.y[~mask][:,self.targetIndex].reshape(1,-1), probs.reshape(1,-1))
        return ranking_loss
        
    def bootStrap(self, target_index, fraction, repeats):
        self.setTarget(target_index)
        self.fraction = fraction

        ranking_losses = list()
        for rep in tqdm_notebook(range(repeats)):
            mask = self.maskNN() #generate a mask for a random block of positives
            mask = self.maskNegs(mask) #mask some negatives too
            
            self.clf.fit(self.x[mask], self.y[mask][:,self.targetIndex])
            ranking_loss = self.evaluateFold(mask)
            ranking_losses.append(ranking_loss)
            if rep>25 and rep%5==0:
                means = [np.mean(ranking_losses[:-i]) for i in np.arange(1,16)]
                diffs = [np.abs(means[-1]- mean) for mean in means[:-1]]
                if max(diffs)<0.0025:
                    break
        return ranking_losses

In [66]:
vsbs_morg = VirtualScreeningBootstrapperMeans(x,y,SGDClassifier(loss='log'))
vsbs_hash = VirtualScreeningBootstrapperMeans(fps_array, y, SGDClassifier(loss='log'))


In [68]:
results_morg = list()
results_hash = list()

fraction=0.1
repeats=100

for targ in np.random.choice(y.shape[1], 10, replace=True):
    ranking_loss = vsbs_morg.bootStrap(targ, fraction, repeats)
    result_morg.append(np.mean(ranking_loss))

    ranking_loss = vsbs_hash.bootStrap(targ, fraction, repeats)
    result_hash.append(np.mean(ranking_loss))

KeyboardInterrupt: 